In [1]:
import numpy as np
import Amazon_Movie_Parser as amp
import Graph_Amazon_Movies as gam
# save numpy array as csv file
from numpy import savetxt
# load numpy array from csv file
from numpy import loadtxt
import networkx as nx
import datetime
import Computations_debug as cmp
from networkx.algorithms import bipartite
import random
import Predict_Movie_v1 as pm
import pickle

In [ ]:
#import matplotlib.pylot as plt

## Normalization: TILE (broadcast) or MATRIX (np.newaxis) (FASTER)

In [ ]:
# Normalization test with TILE
tt = np.array([[2, 4, 6], [1, 5, 8], [20, 4, 6]])
#print(tt.mean(axis=1))
#tt_norm=tt/tt.mean(axis=1)

print('Sample matrix:\n{}'.format(tt))

tt_sum = tt.sum(axis = 1)
print('\nSum of each row:\n{}'.format(tt.sum(axis=1)))

# broadcast with TILE so that we can divide the each element in the row with the same number 
# we will do an element wise division not a dot product here
tt_sum_broadcasted = np.tile(tt_sum, (3, 1)).T
print('\nBroadcasting with TILE:\n{}'.format(tt_sum_broadcasted))

# divide each element in the row with the sum of the row so the sum will be equal to 1
# in other words, it will give the probability/rating for the each cell
tt_norm = tt / tt_sum_broadcasted
print('\nNormalizing with TILE:\n{}'.format(tt_norm))


In [ ]:
# Here, we are testing the normalization using the prepared function, to see if it works as expected
cmp1 = cmp.GraphComp()
tt = np.array([[2, 4, 6], [1, 5, 8], [20, 4, 6]])

#print('Sample matrix:\n{}'.format(tt))

# TILE put in function Normalize_Array()
tt_norm = cmp1.Normalize_Array(tt, axis = 1)
print('\nNormalizing with Normalize_Array function:\n{}'.format(tt_norm))

In [ ]:
# Here, we are testing the normalization using the prepared function, to see if it works as expected
cmp1 = cmp.GraphComp()
tt = np.array([[2, 4, 6], [1, 5, 8], [20, 4, 6]])

#print('Sample matrix:\n{}'.format(tt))

#Normalize_Matrix(self, arr_to_normalize, dim = 'row'):
tt_norm = cmp1.Normalize_Matrix(tt, dim = 'tt')
print('\nNormalizing with Normalize_Matrix function:\n{}'.format(tt_norm))

## Selecting Random Numbers: choices vs sample (UNIQUE)

In [ ]:
tt = [2, 4, 6, 8, 9, 11]
# random.choices 
# random.sample returns unique numbers unlike random.choices 
random.sample(tt, k = 2)

## Convert from Graph to Matrix using Networkx (FASTER than FOR LOOP)

In [ ]:
# Let's see normalization from our dataset to verify that it is working fine
# below that one, you will find a 3x3 matrix simulation, if it would be easier to understand, you can check that one first..
grpComp = cmp.GraphComp()
grp = gam.Graph_Amazon()

# here, there is no Connected or Disconnected check, so, the graph generated will most likele be a DISCONNECTED graph
DISconnected_gr_amazon_movies = grp.Create_Graph(file_name = 'data/movies.txt', n_movies = 8, prs_out = 'dictionary')
# here, we set n_movies = 8 but we have 10 nodes. Why is that?
# Because 8 actually shows the number of edges, not number of the movies (BAD NAMING!)
# So, we have 8 edges and in this 8 edges, we have 10 unique nodes. It might be like 8 users + 2 movies

P_disconnected = nx.to_numpy_matrix(DISconnected_gr_amazon_movies)
print(type(P_disconnected))

print('P_disconnected:\n{}'.format(P_disconnected))

P_disconnected_norm = grpComp.Normalize_Matrix(P_disconnected, dim = 'row')
print('\nP_disconnected-NORMALIZED:\n{}'.format(P_disconnected_norm))

In [ ]:
## Another simulation with a smaller matrix, it migh
tt2 = np.array([[0, 4, 2], [4, 0, 0], [2, 0, 0]])
tt2 = np.asmatrix(tt2)
print(type(tt2))
#tt2_norm = cmp1.Normalize_Matrix(tt2, dim = 'tt2')
tt2_norm = cmp1.Normalize_Matrix(tt2, dim = 'row')

print('Sample matrix:\n{}'.format(tt2))
print('\nSample matrix_NORM:\n{}'.format(tt2_norm))

r_tt = np.zeros(tt2_norm.shape[0])
r_tt[1] = 1
r_tt_zero = r_tt.copy()

print('r at the beginning:\n{}'.format(r_tt_zero))

print('\nLets see how random walk works by selecting the second node_P[1]:'.format(r_tt))
#r = (1 - beta) * np.dot(r, P_norm) + beta * r_zero

# r vectors can be generated in the function but we started to design in that way, sos didn't change it later..
r_tt = grpComp.random_walk_vector(tt2_norm, r_tt, r_tt_zero, beta=0.15, n_steps=1)
print('r - after 1st step:\n{}'.format(r_tt))
# as you can see, after the first strep, the value for r[1]=0.15  and the SUM(row) = 1

r_tt = grpComp.random_walk_vector(tt2_norm, r_tt, r_tt_zero, beta=0.15, n_steps=1)
print('\nr - after 2nd step:\n{}'.format(r_tt))

r_tt = grpComp.random_walk_vector(tt2_norm, r_tt, r_tt_zero, beta=0.15, n_steps=1)
print('\nr - after 3rd step:\n{}'.format(r_tt))

r_tt = grpComp.random_walk_vector(tt2_norm, r_tt, r_tt_zero, beta=0.15, n_steps=1)
print('\nr - after 4th step:\n{}'.format(r_tt))

r_tt = grpComp.random_walk_vector(tt2_norm, r_tt, r_tt_zero, beta=0.15, n_steps=1)
print('\nr - after 5th step:\n{}'.format(r_tt))

# You will easily realize that, for the even number of steps, the probability is high on the user himself,
# on the odd number of steps, the probability is high on the movie he reviewed

In [ ]:
grpComp = cmp.GraphComp()

# *** min numner of reviews is 140 
# if we want to make a connected bipartite graph & if we start picking from review#1
grp = gam.Graph_Amazon()
file_name='data/movies.txt'; n_movies = 140; prs_out='dictionary'

max_connected_gr_amazon_movies = grpComp.Create_Bipartite_Giant_Component(grp, file_name, n_movies, prs_out)

P = nx.to_numpy_matrix(max_connected_gr_amazon_movies)

grpComp.Normalize_Matrix(P, dim = 'row')


## Random Walk Simulation

In [ ]:
r = np.zeros(P_disconnected_norm.shape[0])
r[1] = 1
r_zero = r.copy()

print('r at the beginning:\n{}'.format(r_zero))

print('\nLets see how random walk works by selecting the second node_P[1]:'.format(r))
#r = (1 - beta) * np.dot(r, P_norm) + beta * r_zero

r = grpComp.random_walk_vector(P_disconnected_norm, r, r_zero, beta=0.15, n_steps=1)
print('\nr - after 1st step:\n{}'.format(r))
# as you can see, after the first strep, the value for r[1]=0.15  and the SUM(row) = 1

r = grpComp.random_walk_vector(P_disconnected_norm, r, r_zero, beta=0.15, n_steps=1)
print('\nr - after 2nd step:\n{}'.format(r))

r = grpComp.random_walk_vector(P_disconnected_norm, r, r_zero, beta=0.15, n_steps=1)
print('\nr - after 3rd step:\n{}'.format(r))

## Find Common items in 2 lists with SET (FASTER) - WITHOUT FOR loop

In [ ]:
bottom_nodes = ['u1', 'u2', 'u3', 'u4']
bottom_nodes_v = ['u_v1', 'u2', 'u_v3', 'u4', 'u_v5']
lst_common_users = list(set(bottom_nodes) - (set(bottom_nodes) - set(bottom_nodes_v)))
#lst_common_users = list(set(bottom_nodes_v) - (set(bottom_nodes_v) - set(bottom_nodes)))
print(lst_common_users)

## Create Graph from LIST

In [ ]:
lst_test = [('A141HP4LYPW', 'B003AI2VGA', '3.0'),
     ('B000063W1R', 'A141HP4LYPW', '4.0'),
     ('6304286961', 'A141HP4LYPW', '5.0'),
     ('A141HP4LYPW', '5556167281', '5.0')]

grp1 = gam.Graph_Amazon()
grp_test=grp1.Create_Graph_From_List_WITH_Weight(lst_test)

print(grp_test.nodes)

grp_test_martrix = nx.to_numpy_matrix(grp_test)

print(grp_test_martrix)
# as uyou can see, we have 1 user 4 movies in this set (which is unusual, movies are usually less)
# but this is just an artificial set - you can easily play and test some functions if you would like to now
# i.e. Normalization, Random walk, check networkx functions, etc...

In [ ]:
grp_test.edges.data('weight', default=1)

In [ ]:
grp_test.nodes

In [ ]:
list(grp_test.nodes)[1]

In [ ]:
grp_test.has_edge('A141HP4LYPW', 'B003AI2VGAx')

In [ ]:
gdata = grp_test.get_edge_data('A141HP4LYPW', 'B003AI2VGA')
print(gdata)

In [ ]:
gdata['weight']

In [ ]:
grp_test.get_edge_data('A141HP4LYPW', 'B003AI2VGA')['weight']

In [ ]:
type(grp_test.get_edge_data('A141HP4LYPW', 'B003AI2VGAx'))

In [ ]:
# THIS WAS AN INTERNAL TEST - just keeping as a side note..
# by doing the below conversion, we are just considering the links between the nodes, we ignore the weights
# we try to understand if considering the weights will benefit us or not
#P[P>0]=1

In [ ]:
np.argsort(llt)[0][0][0]

In [ ]:
sorted_nodes=np.argsort(r)

user_similarity = []

# we will not consider the user himself or movies as similar nodes
for idx in sorted_nodes[0][0]:
    if idx != 3 and idx > 0:
        user_similarity.append(idx)
        
user_similarity_top2 = user_similarity[-2:]

print(user_similarity)
print(user_similarity_top2)
print(list(grp_test.nodes)[user_similarity_top2[0]])

## Read Edges from movies file, OUTPUT: Screen

In [ ]:
# the code was quickly written so, it might not be necessarily starting from the 4th review but maybe 5th review
# sensitive calibration haven't done, so index can be +1. 
# but in any case, it gurantees to avoid the overlap (if unlucky, max 1 overlap will be in place)
file_name = 'data/movies.txt'; start_index = 4; n_movies = 5; prs_out = 'screen'

#movie_dict_old = amp.AmazonMovies(n_movies, file_name, prs_out)
movie_dict_old = amp.AmazonMovies_VALIDATION(n_movies, file_name, prs_out, start_index)

In [ ]:

file_name = 'data/movies.txt'; start_index = 2; n_movies = 10; prs_out = 'screen'

movie_dict = amp.AmazonMovies_VALIDATION(n_movies, file_name, prs_out, start_index)

## SEARCH in movies.txt for specific items (FILTER)

In [ ]:
# the search uses 'like' criteria so it search for any string mathcing 
# not a perfect search, just used to debug something -can definitely be improved..
import Amazon_Movie_Parser as amp
file_name = 'data/movies.txt'; n_movies=60000; prs_out = 'screen'
userID = 'AR15V2ULA2EYM';   movieID = ''  #movieID = 'B00005952Q'

amp.Filter_AmazonMovies(n_movies, userID, movieID, file_name, prs_out)

In [ ]:
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

import numpy as np

y_true = [0, 0, 0, 0, 1, 1]
y_pred = [0, 0, 1, 0, 0, 1]

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
lst_tpr = list(tpr); lst_fpr = list(fpr); lst_thresholds = list(thresholds)

print(fpr)
print(tpr)
print(thresholds)

len_tpr = len(lst_tpr)
lst_precision = np.zeros(len_tpr)
lst_recall = np.zeros(len_tpr)
        
for i in range(len(lst_tpr)):
    lst_precision[i] = lst_tpr[i]/(lst_tpr[i] + lst_fpr[i])

roc_auc = roc_auc_score(y_true, y_pred)
print(roc_auc)

precision = precision_score(y_true, y_pred, average='macro') # 'micro'  'weighted' None  average=None, zero_division=1
print(precision)

precision_recall = precision_recall_curve(y_true, y_pred)
print(list(precision_recall))

avg_precision = average_precision_score(y_true, y_pred)
print(avg_precision)

print(lst_sensitivity)

In [ ]:
lst1=[1,2,3]
lst2=[3,5,1]
lst3=[8,2,4]
lstr = [lst1, lst2, lst3]
lstr_2 = []

max_i=0
max_i_val=0
for i, l in enumerate(lstr):
    lstr_2.append(l[2])
    if l[0] > max_i_val:
        max_i_val = l[0]
        max_i = i

print(max_i)
print(max_i_val)
print(lstr[max_i])
print(lstr_2)

In [ ]:
lst4=[8,2,4]
print(lst4)
lst4.remove(8)
print(lst4)

## Mini-Test to Check if any RUN-time Error

In [2]:
file_name = 'data/movies.txt'; n_movies = 5000; n_movies_v = 3000; n_test_users = 20; walk_steps = 2
start_index_v = n_movies; beta = 0.15; top_neighbor = 10; top_N_movie_suggestions = 5

predict_amazon = pm.Amazon()
strt=datetime.datetime.now()
max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL = \
    predict_amazon.create_graphs(file_name, n_movies, n_movies_v, start_index_v)

df_FINAL = \
    predict_amazon.run_validation_TOP_N(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor, top_N_movie_suggestions)
end1=datetime.datetime.now()
diff=end1-strt
df_perf = predict_amazon.save_performance_values(df_FINAL, diff, diff)

params_df = predict_amazon.show_parameters_in_use()
params_df.style.set_properties(**{'text-align': 'left'})

df_perf

*************** 1st Graph calculations - MODEL *************** 
Calculation time-Crete graph from the movies file: 0:00:00.579693
Is connected: True ... Is bipartite: True
BEFORE: # of nodes: 4408 ... # of edges: 4643
Calculation time-1st Part: 0:00:00.742393

*************** 2nd Graph calculations - TEST *************** 
Create_Bipartite_VALIDATION is running...
Calculation time-Crete graph from the movies file: 0:00:00.248519
Is connected: True ... Is bipartite: True
BEFORE: # of nodes: 2487 ... # of edges: 2585
n_reviews for modeling (P matrix) = 61 .. n_movies for testing = 65
n_users for modeling (P matrix) = 4347 .. n_users for testing = 2422
Calculation time-2nd Part: 0:00:00.348293
Preparation starts at 2020-05-23 14:23:42.612856
P to_numpy_matrix: 2020-05-23 14:23:42.725729
P normalize: 2020-05-23 14:23:42.869005
P finalize: 2020-05-23 14:23:42.973682
Calculation time-P matrix generation from the graph: 0:00:00.247952
P_norm is generated: 2020-05-23 14:23:42.986122
Calculation

,AUC,TPR,Precision,FPR,Thresholds,Calc_Time,lst_name,lst_value,other_metrics,metrics_val
0,0.853723,0,0,0,21,Graph Creation,debug_mode,Off,threshold,10
1,,0.166667,0,0,20,0:00:09.739144,n_movies_model,5000,acc,0.95
2,,1,1,0.351064,10,Random Walk & Prediction,n_movies_validation,3000,tpr,0.166667
3,,1,0.153846,1,0,0:00:09.739144,walk_steps,2,fpr,0
4,,,,,,Threshold search,beta,0.15,tnr,1
5,,,,,,0:00:00.158270,top_neighbor,10,fnr,0.833333
6,,,,,,TOTAL TIME,n_test_users,20,precision,1
7,,,,,,0:00:19.636558,threshold,10,f1,0.285714
8,,,,,,,start_index_v,5000,,
9,,,,,,,file_name,data/movies.txt,,


## OTHERS

In [ ]:
top_N_suggestions=10
predict_amazon.df_summary_ratio_v.sort_values(by='ratio_similar', ascending=False).head(top_N_suggestions)

In [ ]:
file_name = 'data/movies.txt'; num_reviews = 5
amp.AmazonMovies(num_reviews, file_name, outputTo='file')

In [ ]:
lst_tst = amp.Read_Connected_Movies('Movies_22.5.2020_20.47.56.txt')
print(lst_tst)
arr_tst = lst_tst[2][0].replace('[','').replace(']','').split(sep="' '")
#print(len(lst_tst))
print(arr_tst)
print(arr_tst[0])

In [ ]:
#import pickle
file_name = 'data/movies.txt'; num_reviews = 75000

amp.AmazonMovies(num_reviews, file_name, outputTo='dict_to_file')


In [ ]:
xx = amp.Load_Pickle_File_VAL('Movies_22.5.2020_23.21.10__100_reviews.pkl', 10, 5)

In [ ]:
xx

In [ ]:
import pickle
out = dict(list(xx.items())[2: 7])
out

In [ ]:
type(out)

In [ ]:
lst_1=[1]
lst_2=[2,3,1]
lst_1+=lst_2
print(lst_1)

In [ ]:
lst_1.count(2)